In [58]:
import numpy as np
import cv2
import json

In [187]:
# load image
image = cv2.imread('C:/Users/lacto/Documents/GitHub/HarmoniesRender/training_boards/screenshots/Board5.png')

# get ref grid
f = open('hex_positions.json')
hex_centers = json.load(f)

In [188]:
# convert to gray and blur
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.blur(image, (10,10))

# detect edges
edges = cv2.Canny(gray, 30, 150)
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# crop to game board
largest_contour = max(contours, key=cv2.contourArea)
x, y, w, h = cv2.boundingRect(largest_contour)
board_region = image[y:y+h, x:x+w] 

In [189]:
cv2.imshow('', board_region)
cv2.waitKey(0)

-1

In [190]:
cv2.imwrite('C:/Users/lacto/Documents/GitHub/HarmoniesRender/training_boards/cropped_Board5.jpg', board_region)

True

In [150]:
# Define token and cube colors (RGB)
color_ranges = {
    "red": ((158, 37, 45), (253, 131, 122)),  
    "blue": ((29, 118, 127), (126, 223, 236)),
    "green": ((126, 130, 23), (211, 222, 103)),
    "yellow": ((202, 167, 25), (255, 231, 103)),
    "gray": ((109, 109, 109), (183, 180, 178)),  
    "brown": ((132, 82, 47), (147, 95, 59)),
    "board":((223, 202, 168), (234, 214, 185)),
    "spirit": ((159, 127, 88), (252, 248, 242)),
    "animal": ((198, 63, 28), (255, 156, 92))
}

In [136]:
# token & cube detection
rgb = cv2.cvtColor(board_region, cv2.COLOR_BGR2RGB)

# region of interest (ROI) parameters
roi_w = 10
roi_l = 25

# Extract ROI for each hex
for (hex_x, hex_y) in hex_centers:
    roi = rgb[hex_y - roi_l : hex_y + roi_l, hex_x - roi_w : hex_x + roi_w]  # Adjust based on hex size

    # Apply color masks to count token pixels
    for token_type, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(roi, np.array(lower), np.array(upper))
        pixel_count = cv2.countNonZero(mask)

        if pixel_count > 50:  # Threshold to ignore small noise
            detected_tokens.append(token_type)

    # Store detected tokens
    board_state["tokens"].append({
        "x": hex_x, "y": hex_y,
        "stack": len(detected_tokens),
        "types": detected_tokens
    })

In [138]:
roi_w = 10
roi_l = 25

for (hex_x, hex_y) in hex_centers:
    top_left = (hex_x - roi_w, hex_y - roi_l)
    bottom_right = (hex_x + roi_w, hex_y + roi_l)
    cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)  # Green rectangle for debugging


cv2.imshow('', image)
cv2.waitKey(0)

-1

In [137]:
print(board_state['tokens'][0])

{'x': 27, 'y': 25, 'stack': 1, 'types': ['gray', 'red', 'gray', 'yellow', 'gray', 'red', 'gray', 'red', 'gray', 'blue', 'gray', 'red', 'gray', 'red', 'gray', 'red', 'blue', 'gray', 'blue', 'gray', 'red', 'blue', 'gray', 'red', 'blue', 'gray', 'blue', 'gray', 'green', 'yellow', 'brown', 'red', 'gray', 'red', 'gray', 'gray', 'brown', 'yellow', 'gray', 'red', 'gray', 'red', 'gray', 'yellow', 'gray', 'red', 'gray', 'red', 'gray', 'blue', 'gray', 'red', 'gray', 'red', 'gray', 'red', 'blue', 'gray', 'blue', 'gray', 'red', 'blue', 'gray', 'red', 'blue', 'gray', 'blue', 'gray', 'green', 'yellow', 'brown', 'red', 'gray', 'red', 'gray', 'gray', 'brown', 'yellow', 'gray', 'red', 'gray', 'red', 'gray', 'yellow', 'gray', 'red', 'gray', 'red', 'gray', 'blue', 'gray', 'red', 'gray', 'red', 'gray', 'red', 'blue', 'gray', 'blue', 'gray', 'red', 'blue', 'gray', 'red', 'blue', 'gray', 'blue', 'gray', 'green', 'yellow', 'brown', 'red', 'gray', 'red', 'gray', 'gray', 'brown', 'yellow', 'gray', 'red', 'gray

In [ ]:
def detect_top_colors(roi, step=10):
    """
    Detects the primary colors in an ROI, scanning from top to bottom in horizontal slices.
    - step: The vertical pixel step size to reduce redundant detections.
    """
    detected_tokens = []
    roi_height = roi.shape[0]
    
    for y in range(0, roi_height, step):  # Scan in steps to avoid repeated detections
        row = roi[y:y+step, :]  # Extract horizontal slice
        for token_type, (lower, upper) in color_ranges.items():
            mask = cv2.inRange(row, np.array(lower), np.array(upper))
            if cv2.countNonZero(mask) > STACK_THRESHOLD:
                if token_type not in detected_tokens:
                    detected_tokens.append(token_type)  # Keep order of detection
                    break  # Stop once a color is found in this row

    return detected_tokens

def detect_stack_levels(roi):
    """
    Determines how many stacked tokens are in the hex by analyzing vertical color changes.
    """
    roi_height = roi.shape[0]
    detected_stacks = []
    prev_color = None

    for y in range(roi_height):
        row = roi[y:y+1, :]  # Single-pixel row scan
        for token_type, (lower, upper) in color_ranges.items():
            mask = cv2.inRange(row, np.array(lower), np.array(upper))
            if cv2.countNonZero(mask) > STACK_THRESHOLD:
                if token_type != prev_color:  # Register color change
                    detected_stacks.append(token_type)
                    prev_color = token_type
                break  # Stop checking after first detected color in this row

    return len(detected_stacks), detected_stacks  # Number of detected stack levels


def detect_cube(roi):
    """
    Checks the top part of the ROI to detect if a cube is placed on top.
    """
    cube_roi = roi[0:int(roi.shape[0] * 0.2), :]  # Top 20% of ROI
    cube_mask = cv2.inRange(cube_roi, np.array(color_ranges["cube"][0]), np.array(color_ranges["cube"][1]))
    
    return cv2.countNonZero(cube_mask) > CUBE_THRESHOLD  # Returns True if cube detected


In [ ]:
roi_w = 10
roi_l = 25

for (hex_x, hex_y) in hex_centers:
    roi = hsv[hex_y - roi_l : hex_y + roi_l, hex_x - roi_w : hex_x + roi_w]  # Extract region

    # Get stack details
    stack_count, detected_tokens = detect_stack_levels(roi)

    # Get cube presence
    has_cube = detect_cube(roi)

    board_state["tokens"].append({
        "x": hex_x, "y": hex_y,
        "stack": stack_count,
        "types": detected_tokens,
        "cube": has_cube
    })


In [151]:
for (hex_x, hex_y) in hex_centers:
    roi = rgb[hex_y - roi_l : hex_y + roi_l, hex_x - roi_w : hex_x + roi_w]

    print(f"\nHex at ({hex_x}, {hex_y})")

    for token_type, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(roi, np.array(lower), np.array(upper))
        pixel_count = cv2.countNonZero(mask)

        # Special case for spirit color: subtract board pixels
        if token_type == "spirit":
            board_mask = cv2.inRange(roi, np.array(color_ranges["board"][0]), np.array(color_ranges["board"][1]))
            spirit_adjusted_count = max(0, pixel_count - cv2.countNonZero(board_mask))  # Ensure non-negative

            print(f"  spirit (raw): {pixel_count} pixels")
            print(f"  spirit (adjusted): {spirit_adjusted_count} pixels (spirit - board)")
        else:
            print(f"  {token_type}: {pixel_count} pixels")



Hex at (27, 25)
  red: 0 pixels
  blue: 0 pixels
  green: 0 pixels
  yellow: 0 pixels
  gray: 430 pixels
  brown: 0 pixels
  board: 118 pixels
  spirit (raw): 309 pixels
  spirit (adjusted): 191 pixels (spirit - board)
  animal: 0 pixels

Hex at (27, 75)
  red: 80 pixels
  blue: 0 pixels
  green: 3 pixels
  yellow: 12 pixels
  gray: 212 pixels
  brown: 2 pixels
  board: 13 pixels
  spirit (raw): 71 pixels
  spirit (adjusted): 58 pixels (spirit - board)
  animal: 128 pixels

Hex at (27, 125)
  red: 0 pixels
  blue: 0 pixels
  green: 10 pixels
  yellow: 332 pixels
  gray: 72 pixels
  brown: 0 pixels
  board: 264 pixels
  spirit (raw): 461 pixels
  spirit (adjusted): 197 pixels (spirit - board)
  animal: 4 pixels

Hex at (27, 175)
  red: 76 pixels
  blue: 0 pixels
  green: 8 pixels
  yellow: 8 pixels
  gray: 221 pixels
  brown: 0 pixels
  board: 279 pixels
  spirit (raw): 419 pixels
  spirit (adjusted): 140 pixels (spirit - board)
  animal: 124 pixels

Hex at (27, 225)
  red: 79 pixels
 